In [147]:
import pandas as pd
from scipy import sparse

In [148]:
ratings = pd.read_csv('dataset/ratings.csv')
movies = pd.read_csv('dataset/movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(745, 4)


,movieId,title,userId,rating
0,1,The Raid Redemption,1,4.0
1,1,The Raid Redemption,5,4.0
2,1,The Raid Redemption,7,4.5
3,1,The Raid Redemption,15,2.5
4,1,The Raid Redemption,17,4.5


In [149]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (346, 10)
After:  (346, 8)


In [150]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(1000)

title,Ada Apa Dengan Cinta? 2,Dilan 1991 (2019),Dua Garis Biru (2019),Mangkujiwo,Milea: Suara dari Dilan (2020),Pengabdi Setan (2017),The Raid Redemption,Warkop DKI Reborn
title,,,,,,,,
Ada Apa Dengan Cinta? 2,1.000000,0.001824,0.099450,0.016459,0.134260,-0.095656,-0.131563,0.028879
Dilan 1991 (2019),0.001824,1.000000,0.135350,0.390909,0.115845,0.285695,0.050939,0.328235
Dua Garis Biru (2019),0.099450,0.135350,1.000000,0.054981,0.009191,0.126235,-0.062541,0.111729
Mangkujiwo,0.016459,0.390909,0.054981,1.000000,0.028505,0.092928,-0.023670,0.306006
Milea: Suara dari Dilan (2020),0.134260,0.115845,0.009191,0.028505,1.000000,-0.082493,-0.019961,0.103760
Pengabdi Setan (2017),-0.095656,0.285695,0.126235,0.092928,-0.082493,1.000000,0.118524,0.251626
The Raid Redemption,-0.131563,0.050939,-0.062541,-0.023670,-0.019961,0.118524,1.000000,0.020708
Warkop DKI Reborn,0.028879,0.328235,0.111729,0.306006,0.103760,0.251626,0.020708,1.000000


In [151]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [152]:
romantic_lover = [("The Raid Redemption",4),("Warkop DKI Reborn",3),("Dilan 1991 (2019)",2),("Dua Garis Biru (2019)",5)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

,Ada Apa Dengan Cinta? 2,Dilan 1991 (2019),Dua Garis Biru (2019),Mangkujiwo,Milea: Suara dari Dilan (2020),Pengabdi Setan (2017),The Raid Redemption,Warkop DKI Reborn
0,-0.197344,0.076409,-0.093812,-0.035505,-0.029942,0.177786,1.500000,0.031061
1,0.014439,0.164118,0.055864,0.153003,0.051880,0.125813,0.010354,0.500000
2,-0.000912,-0.500000,-0.067675,-0.195454,-0.057922,-0.142848,-0.025470,-0.164118
3,0.248625,0.338376,2.500000,0.137453,0.022978,0.315587,-0.156353,0.279321


In [153]:
similar_movies.sum().sort_values(ascending=False).head(10)

Dua Garis Biru (2019)             2.394377
The Raid Redemption               1.328531
Warkop DKI Reborn                 0.646265
Pengabdi Setan (2017)             0.476339
Dilan 1991 (2019)                 0.078902
Ada Apa Dengan Cinta? 2           0.064808
Mangkujiwo                        0.059497
Milea: Suara dari Dilan (2020)   -0.013007
dtype: float64

In [154]:
action_lover = [("The Raid Redemption",5),("Warkop DKI Reborn",4),("Dilan 1991 (2019)",3),("Dua Garis Biru (2019)",1)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

The Raid Redemption               2.650343
Warkop DKI Reborn                 1.548294
Dilan 1991 (2019)                 0.916675
Pengabdi Setan (2017)             0.627245
Mangkujiwo                        0.512818
Milea: Suara dari Dilan (2020)    0.149872
Ada Apa Dengan Cinta? 2          -0.433852
Dua Garis Biru (2019)            -1.421085
dtype: float64